In [ ]:
import numpy as np
# import matplotlib
# matplotlib.use('QtAgg')
import matplotlib.pyplot as plt
import pandas as pd
import timeit
import time
import scienceplots

plt.style.use('science')

In [ ]:
a = 1
x_0 = a/2
sigma = a/10
t_max = 0.2

N_default = 100
x = np.linspace(0, a, 3*N_default)
t = np.linspace(0, t_max, 3*N_default)
T_0 = np.exp(-(x - x_0)**2/(sigma**2))
T_0[0] = 0
T_0[-1] = 0

h_x = x[1] - x[0]
h_t = t[1] - t[0]


def fourierer(T_0, t, h_x, h_t, D=0.5):
    N = 2 * len(T_0)
    T_0 = np.concatenate((-T_0, T_0))
    t = np.linspace(0, 2*np.max(t), N)
    T_new = np.zeros((N, N), dtype=complex)
    fft_T_0 = np.fft.fft(T_0)
    k = np.fft.fftfreq(N, d=h_x)
    for i in range(N):
        T_new[:, i] = np.exp(-4. * D * t * np.pi**2. * k[i]**2.) * fft_T_0[i]
    for j in range(N):
        T_new[j, :] = np.fft.ifft(T_new[j, :])
    T_new = T_new[:N//2,N//2:]
    T_new = T_new.real
    return T_new

solut = fourierer(T_0, t, h_x, h_t)
x, t = np.meshgrid(x, t)
fig = plt.figure()
ax = plt.axes(projection='3d')
surf = ax.plot_surface(x, t, solut, cmap='viridis')
ax.set_xlim(0, a)
ax.set_zlim(0, 1)
ax.set_ylim(0, t_max)
ax.set_title('Homogeni robni pogoji, Fourierova metoda')
ax.tick_params(labelbottom=True, labelleft=True, labelright=False)
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$T$')
ax.view_init(15, 60)
cb = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=15)
cb.set_label('$T$')
cb.set_ticks([0.0, 0.4, 0.8])
plt.tight_layout()
plt.savefig('graphs/homf_gauss.pdf', dpi=512)
plt.clf()

<Figure size 350x262.5 with 0 Axes>

In [ ]:
a = 1
x_0 = a/2
sigma = a/10
t_max = 0.2

N_default = 100
x = np.linspace(0, a, 3*N_default)
t = np.linspace(0, t_max, 3*N_default)
T_0 = np.exp(-(x - x_0)**2/(sigma**2))
T_0[0] = 0
T_0[-1] = 0

h_x = x[1] - x[0]
h_t = t[1] - t[0]

def fourierer(T_0, t, h_x, h_t, D=0.5):
    N = len(T_0)
    T_new = np.zeros((N, N), dtype=complex)
    fft_T_0 = np.fft.fft(T_0)
    k = np.fft.fftfreq(N, d=h_x)
    for i in range(N):
        T_new[:, i] = np.exp(-4. * D * t * np.pi**2. * k[i]**2.) * fft_T_0[i]
    for j in range(N):
        T_new[j, :] = np.fft.ifft(T_new[j, :])
    T_new = T_new.real
    return T_new

soluty = fourierer(T_0, t, h_x, h_t)
x, t = np.meshgrid(x, t)
fig = plt.figure()
ax = plt.axes(projection='3d')
surf = ax.plot_surface(x, t, soluty, cmap='viridis')
ax.set_xlim(0, a)
ax.set_zlim(0, 1)
ax.set_ylim(0, t_max)
ax.set_title('Periodični robni pogoji, Fourierova metoda')
ax.tick_params(labelbottom=True, labelleft=True, labelright=False)
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$T$')
ax.view_init(15, 60)
cb = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=15)
cb.set_label('$T$')
cb.set_ticks([0.0, 0.4, 0.8])
plt.tight_layout()
plt.savefig('graphs/perf_gauss.pdf', dpi=512)
plt.clf()

<Figure size 350x262.5 with 0 Axes>

In [4]:
def B_k(delta, k, x0):
    a1 = x0 - delta * (k - 2)
    a2 = x0 - delta * (k - 1)
    # a3 = x0 - delta * k
    a4 = x0 - delta * (k + 1)
    a5 = x0 - delta * (k + 2)

    if x0 <= delta * (k - 2):
        return 0.0
    elif x0 <= delta * (k - 1):
        return (a1 ** 3) / (delta ** 3)
    elif x0 <= delta * k:
        return (a1 ** 3) / (delta ** 3) - 4 * (a2 ** 3) / (delta ** 3)
    elif x0 <= delta * (k + 1):
        return 4 * (a4 ** 3) / (delta ** 3) - (a5 ** 3) / (delta ** 3)
    elif x0 <= delta * (k + 2):
        return -(a5 ** 3) / (delta ** 3)
    else:
        return 0.0

def spline_sol(x_0, t, x, D=0.5, a=1.):
    t1 = time.time()
    N = len(x)
    t_max = t[-1]
    dx = a/N
    dt = t_max/N

    main_diagonal_A = 4 * np.ones(N)
    off_diagonal_A = np.ones(N - 1)
    A = np.diag(main_diagonal_A, 0) + np.diag(off_diagonal_A, -1) + np.diag(off_diagonal_A, 1)

    main_diagonal_B = -2 * np.ones(N)
    off_diagonal_B = np.ones(N - 1)
    B = 6 * D / dx**2 * (np.diag(main_diagonal_B, 0) + np.diag(off_diagonal_B, -1) + np.diag(off_diagonal_B, 1))

    C = np.linalg.inv(A - dt / 2 * B) @ (A + dt / 2 * B)

    c0 = np.linalg.solve(A, x_0)
    
    T = []
    for t_ in t:
        if t_ == 0:
            c = c0
        c = C@c
        T_xs = []
        for xi in range(len(x)):
            T_x = 0
            for k in range(xi-1,N+2):
                if k == -1:
                    c_k = -c[1]
                elif k == 0 or k == N:
                    c_k = 0
                elif k == N+1:
                    c_k = -c[N-1]
                else:
                    c_k = c[k]
                T_x += c_k*B_k(dx, k, x[xi])
            T_xs.append(T_x)
        T.append(T_xs)
    T = np.array(T)
    T = np.concatenate((np.array([x_0]), T[:-1]))
    dt = time.time() - t1
    return T, dt

In [ ]:
a = 1
x_0 = a/2
sigma = a/10
t_max = 0.2

N_default = 100
x = np.linspace(0, a, 3*N_default)
t = np.linspace(0, t_max, 3*N_default)
T_0 = np.exp(-(x - x_0)**2/(sigma**2))

h_x = np.diff(x)[0]
h_t = np.diff(t)[0]

T_0[0] = 0
T_0[-1] = 0

sol = spline_sol(T_0, t, x)
x, t = np.meshgrid(x, t)
fig = plt.figure()
ax = plt.axes(projection='3d')
surf = ax.plot_surface(x, t, sol[0], cmap='viridis')
ax.set_xlim(0, a)
ax.set_zlim(0, 1)
ax.set_ylim(0, t_max)
ax.set_title('Homogen robni pogoj, Metoda končnih elementov')
ax.tick_params(labelbottom=True, labelleft=True, labelright=False)
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$T$')
ax.view_init(15, 60)
cb = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=15)
cb.set_label('$T$')
cb.set_ticks([0.0, 0.4, 0.8])
plt.tight_layout()
plt.savefig('graphs/homp_gauss.pdf', dpi=512)
plt.clf()

<Figure size 350x262.5 with 0 Axes>

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
surf = ax.plot_surface(x, t, np.abs(sol[0] - solut), cmap='viridis')
ax.set_xlim(0, a)
# ax.set_zlim(0, 1)
ax.set_ylim(0, t_max)
ax.set_title('Abs. razlika med Fourierovo metodo in metodo končnih elementov')
ax.tick_params(labelbottom=True, labelleft=True, labelright=False)
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$T$')
ax.view_init(15, 60)
cb = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)
cb.set_label('$T$')
# cb.set_ticks([0.0, 0.4, 0.8])
plt.tight_layout()
plt.savefig('graphs/diff_gauss.pdf', dpi=512)
plt.clf()

<Figure size 350x262.5 with 0 Axes>

In [ ]:
import psutil
import tracemalloc
from tqdm import tqdm
def measure(func, *args):
    tracemalloc.start()
    start_time = time.time()
    _result = func(*args)
    end_time = time.time()
    _, memory_usage = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    execution_time = end_time - start_time
    
    # Measure memory usage
    # process = psutil.Process()
    # memory_info = process.memory_info()
    # memory_usage = memory_info.rss / 1024 / 1024  # in MB
    memory_usage = memory_usage /1024 /1024
    
    return execution_time, memory_usage

a = 1
x_0 = a/2
sigma = a/10
t_max = 0.3

spline_res = []
fourier_res = []

# import gc

NS = range(10, 140, 25)
for N_default in tqdm(NS):
    x = np.linspace(0, a, 3*N_default)
    t = np.linspace(0, t_max, 3*N_default)
    T_0 = np.exp(-(x - x_0)**2/(sigma**2))

    h_x = np.diff(x)[0]
    h_t = np.diff(t)[0]

    T_0[0] = 0
    T_0[-1] = 0

    # gc.collect()
    time2, memory2 = measure(fourierer, T_0, t, h_x, h_t)
    fourier_res.append((time2, memory2))


    # gc.collect()
    time_, memory_ = measure(spline_sol, T_0, t, x)
    spline_res.append((time_, memory_))

# Plot execution time
plt.plot(NS, [res[0] for res in spline_res], label='Spline')
plt.plot(NS, [res[0] for res in fourier_res], label='Fourier')
plt.xlabel('N')
plt.ylabel('Časovna zahtevnost [s]')
plt.legend(
    facecolor='white',    # background color
    edgecolor='black',    # border color
    framealpha=0.8,       # transparency (1 is opaque)
    frameon=True          # ensure the frame is drawn
)
plt.grid()
plt.yscale('log')
plt.savefig('graphs/time_gauss.pdf', dpi=512)
plt.clf()

# Plot memory usage
plt.plot(NS, [res[1] for res in spline_res], label='Spline')
plt.plot(NS, [res[1] for res in fourier_res], label='Fourier')
plt.xlabel('N')
plt.ylabel('Poraba pomnilnika [MB]')
plt.legend(
    facecolor='white',    # background color
    edgecolor='black',    # border color
    framealpha=0.8,       # transparency (1 is opaque)
    frameon=True          # ensure the frame is drawn
)
plt.grid()
plt.yscale('log')
plt.savefig('graphs/mem_gauss.pdf', dpi=512)
plt.clf()


100%|██████████| 6/6 [06:12<00:00, 62.15s/it] 


<Figure size 350x262.5 with 0 Axes>

In [ ]:
a = 1
x_0 = a/2
sigma = a/10
t_max = 0.2

N_default = 100
x = np.linspace(0, a, 3*N_default)
t = np.linspace(0, t_max, 3*N_default)
T_0 = np.sin(np.pi*x/a)
T_0[0] = 0
T_0[-1] = 0

h_x = x[1] - x[0]
h_t = t[1] - t[0]


def fourierer(T_0, t, h_x, h_t, D=0.5):
    N = 2 * len(T_0)
    T_0 = np.concatenate((-T_0, T_0))
    t = np.linspace(0, 2*np.max(t), N)
    T_new = np.zeros((N, N), dtype=complex)
    fft_T_0 = np.fft.fft(T_0)
    k = np.fft.fftfreq(N, d=h_x)
    for i in range(N):
        T_new[:, i] = np.exp(-4. * D * t * np.pi**2. * k[i]**2.) * fft_T_0[i]
    for j in range(N):
        T_new[j, :] = np.fft.ifft(T_new[j, :])
    T_new = T_new[:N//2,N//2:]
    T_new = T_new.real
    return T_new

solf = fourierer(T_0, t, h_x, h_t)
x, t = np.meshgrid(x, t)
fig = plt.figure()
ax = plt.axes(projection='3d')
surf = ax.plot_surface(x, t, solf, cmap='viridis')
ax.set_xlim(0, a)
ax.set_zlim(0, 1)
ax.set_ylim(0, t_max)
ax.set_title('Homogeni robni pogoji, Fourierova metoda')
ax.tick_params(labelbottom=True, labelleft=True, labelright=False)
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$T$')
ax.view_init(15, 60)
cb = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=15)
cb.set_label('$T$')
cb.set_ticks([0.0, 0.4, 0.8])
plt.tight_layout()
plt.savefig('graphs/homf_sin.pdf', dpi=512)
plt.clf()

<Figure size 350x262.5 with 0 Axes>

In [ ]:
a = 1
x_0 = a/2
sigma = a/10
t_max = 0.2

N_default = 100
x = np.linspace(0, a, 3*N_default)
t = np.linspace(0, t_max, 3*N_default)
T_0 = np.sin(np.pi*x/a)

h_x = np.diff(x)[0]
h_t = np.diff(t)[0]

T_0[0] = 0
T_0[-1] = 0

sol = spline_sol(T_0, t, x)
x, t = np.meshgrid(x, t)
fig = plt.figure()
ax = plt.axes(projection='3d')
surf = ax.plot_surface(x, t, sol[0], cmap='viridis')
ax.set_xlim(0, a)
ax.set_zlim(0, 1)
ax.set_ylim(0, t_max)
ax.set_title('Homogeni robni pogoji, Fourierova metoda')
ax.tick_params(labelbottom=True, labelleft=True, labelright=False)
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$T$')
ax.view_init(15, 60)
cb = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=15)
cb.set_label('$T$')
cb.set_ticks([0.0, 0.4, 0.8])
plt.tight_layout()
plt.savefig('graphs/perf_sin.pdf', dpi=512)
plt.clf()

<Figure size 350x262.5 with 0 Axes>

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
surf = ax.plot_surface(x, t, np.abs(sol[0] - solf), cmap='viridis')
ax.set_xlim(0, a)
# ax.set_zlim(0, 1)
ax.set_ylim(0, t_max)
ax.set_title('Abs. razlika med Fourierovo metodo in metodo končnih elementov')
ax.tick_params(labelbottom=True, labelleft=True, labelright=False)
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$T$')
ax.view_init(15, 60)
cb = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)
cb.set_label('$T$')
# cb.set_ticks([0.0, 0.4, 0.8])
plt.tight_layout()
plt.savefig('graphs/diff_sin.pdf', dpi=512)
plt.clf()

<Figure size 350x262.5 with 0 Axes>

In [ ]:
import psutil
import tracemalloc
from tqdm import tqdm
def measure(func, *args):
    tracemalloc.start()
    start_time = time.time()
    _result = func(*args)
    end_time = time.time()
    _, memory_usage = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    execution_time = end_time - start_time
    
    # Measure memory usage
    # process = psutil.Process()
    # memory_info = process.memory_info()
    # memory_usage = memory_info.rss / 1024 / 1024  # in MB
    memory_usage = memory_usage /1024 /1024
    
    return execution_time, memory_usage

a = 1
x_0 = a/2
sigma = a/10
t_max = 0.3

spline_res = []
fourier_res = []

# import gc

NS = range(10, 140, 25)
for N_default in tqdm(NS):
    x = np.linspace(0, a, 3*N_default)
    t = np.linspace(0, t_max, 3*N_default)
    T_0 = np.exp(-(x - x_0)**2/(sigma**2))

    h_x = np.diff(x)[0]
    h_t = np.diff(t)[0]

    T_0[0] = 0
    T_0[-1] = 0

    # gc.collect()
    time2, memory2 = measure(fourierer, T_0, t, h_x, h_t)
    fourier_res.append((time2, memory2))


    # gc.collect()
    time_, memory_ = measure(spline_sol, T_0, t, x)
    spline_res.append((time_, memory_))

# Plot execution time
plt.plot(NS, [res[0] for res in spline_res], label='Spline')
plt.plot(NS, [res[0] for res in fourier_res], label='Fourier')
plt.xlabel('N')
plt.ylabel('Časovna zahtevnost [s]')
plt.legend(
    facecolor='white',    # background color
    edgecolor='black',    # border color
    framealpha=0.8,       # transparency (1 is opaque)
    frameon=True          # ensure the frame is drawn
)
plt.grid()
plt.yscale('log')
plt.savefig('graphs/time_sin.pdf', dpi=512)
plt.clf()

# Plot memory usage
plt.plot(NS, [res[1] for res in spline_res], label='Spline')
plt.plot(NS, [res[1] for res in fourier_res], label='Fourier')
plt.xlabel('N')
plt.ylabel('Poraba pomnilnika [MB]')
plt.legend(
    facecolor='white',    # background color
    edgecolor='black',    # border color
    framealpha=0.8,       # transparency (1 is opaque)
    frameon=True          # ensure the frame is drawn
)
plt.grid()
plt.yscale('log')
plt.savefig('graphs/mem_sin.pdf', dpi=512)
plt.clf()

100%|██████████| 6/6 [06:13<00:00, 62.20s/it] 


<Figure size 350x262.5 with 0 Axes>